In [7]:
import pandas as pd

# Replace 'b1/your_file.xlsx' with the correct path
file_path = "IMF.xlsx"

# Read the first sheet of the Excel file
df = pd.read_excel(file_path, engine="openpyxl")  

# Display the first few rows
print(df.head())


  International Financial Statistics (IFS)  \
0                                      NaN   
1                                   France   
2                                   France   
3                                   France   
4                                   France   

                                          Unnamed: 1   Unnamed: 2 Unnamed: 3  \
0                                                NaN  1959.000000   Dec 1959   
1    Economic Activity, Industrial Production, Index    32.250545  35.365725   
2  Exchange Rates, National Currency Per U.S. Dol...     4.937060    4.93706   
3  International Reserves and Liquidity, Reserves...  1735.740000    1735.74   
4     Prices, Consumer Price Index, All items, Index    10.018821  10.208981   

    Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ...  \
0  1960.000000   Jan 1960   Feb 1960   Mar 1960   Apr 1960   May 1960  ...   
1    35.172258  33.997257  34.793828  34.916378  35.712949  35.743586  ...   
2     4.

In [14]:
print(df.info())  # Get data types and non-null counts
print(df.columns)  # Show column names


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 407 entries, 0 to 406
dtypes: float64(32), object(375)
memory usage: 31.9+ KB
None
Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       397, 398, 399, 400, 401, 402, 403, 404, 405, 406],
      dtype='int64', length=407)


In [15]:
print(df.iloc[:, 0].head(10))  # Print the first 10 rows of the first column


0    International Financial Statistics (IFS)
1                                         NaN
2                                      France
3                                      France
4                                      France
5                                      France
6                               United States
7                               United States
8                               United States
9                               United States
Name: 0, dtype: object


# construct variable

## monthly growth in the nominal exchange rate

In [18]:
import pandas as pd

# Load the dataset
file_path = "IMF.xlsx"  # Update this with your actual file path
df = pd.read_excel(file_path, header=None, engine="openpyxl")

# Find the row index where "Exchange Rates, National Currency Per U.S. Dollar" appears
exchange_rate_row = df[df.iloc[:, 1].str.contains("Exchange Rates", na=False)].index[0]

# Extract the relevant row (France's exchange rate data)
exchange_rate_data = df.iloc[exchange_rate_row, 1:].reset_index(drop=True)

# Convert values to numeric (ignore the first value, which is text)
exchange_rate_data = pd.to_numeric(exchange_rate_data[1:], errors="coerce")

# Drop NaN values
exchange_rate_data.dropna(inplace=True)


In [22]:
# Compute the monthly growth rate in percentage
monthly_growth = exchange_rate_data.pct_change() * 100

# Convert to DataFrame for better visualization
monthly_growth_df = pd.DataFrame({
    "Month": range(1, len(monthly_growth) + 1),
    "Growth (%)": monthly_growth.values
})

# Display the first few rows
print(monthly_growth_df)


     Month  Growth (%)
0        1         NaN
1        2    0.000000
2        3    0.000000
3        4    0.000000
4        5    0.000000
..     ...         ...
400    401   -4.134170
401    402   -0.295841
402    403   -2.900618
403    404   -2.047013
404    405    1.317868

[405 rows x 2 columns]


## real exchange rate

In [25]:
# Find row indices dynamically
nominal_exchange_row = df[df.iloc[:, 0].str.contains("Exchange Rates", case=False, na=False)].index[0]
france_cpi_row = df[df.iloc[:, 0].str.contains("Prices, Consumer Price Index", case=False, na=False) & df.iloc[:, 1].str.contains("France", na=False)].index[0]
us_cpi_row = df[df.iloc[:, 0].str.contains("Prices, Consumer Price Index", case=False, na=False) & df.iloc[:, 1].str.contains("United States", na=False)].index[0]

# Extract the monthly data (skip the row label)
nominal_exchange_rate = pd.to_numeric(df.iloc[nominal_exchange_row, 1:], errors="coerce")
france_cpi = pd.to_numeric(df.iloc[france_cpi_row, 1:], errors="coerce")
us_cpi = pd.to_numeric(df.iloc[us_cpi_row, 1:], errors="coerce")


IndexError: index 0 is out of bounds for axis 0 with size 0